In [7]:
# Installation de la librairie ultralytics (si pas déjà fait)
%pip install ultralytics

import torch
from ultralytics import YOLO

# Vérification du GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Entraînement sur : {device}")
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Note: you may need to restart the kernel to use updated packages.
Entraînement sur : cuda
NVIDIA GeForce RTX 4070 Laptop GPU


In [10]:
# Chargement du modèle nano pré-entraîné (transfer learning)
# Le fichier sera téléchargé automatiquement s'il n'existe pas
import os

# Si YOLOv11 n'est pas dans models/yolo11n_base/
if 'yolo11n.pt' not in os.listdir('models/yolo11n_base/'):
    model = YOLO('yolo11n.pt')
    print("Modèle YOLOv11 nano téléchargé et chargé.")
else:
    model = YOLO('models/yolo11n_base/yolo11n.pt')
    print("Modèle YOLOv11 nano chargé depuis le répertoire local.")

# Chemin vers votre fichier de configuration créé précédemment
# Assurez-vous que le chemin est correct par rapport à l'endroit où vous lancez le notebook
yaml_path = 'datasets/red_cube/data.yaml'

Modèle YOLOv11 nano chargé depuis le répertoire local.


In [12]:
import os

# 1. Récupération du chemin absolu de votre projet actuel
project_root = os.getcwd()
dataset_abs_path = os.path.join(project_root, "datasets", "red_cube")

print(f"Chemin absolu détecté : {dataset_abs_path}")

# 2. Contenu du fichier YAML avec le chemin absolu 'path'
# Cela empêche YOLO d'aller chercher dans les dossiers par défaut ou le cache
yaml_content = f"""
path: {dataset_abs_path} # Chemin absolu (fixe le problème)
train: train/images      # Relatif à 'path'
val: valid/images        # Relatif à 'path'

nc: 1
names: ['red_cube']
"""

# 3. Écriture du fichier data.yaml correct
yaml_file_path = os.path.join(dataset_abs_path, "data.yaml")
with open(yaml_file_path, "w") as f:
    f.write(yaml_content)

print(f"✅ Fichier data.yaml réparé ici : {yaml_file_path}")

Chemin absolu détecté : /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube
✅ Fichier data.yaml réparé ici : /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/data.yaml


In [ ]:
print("Démarrage du fine-tuning...")

# Lancement de l'entraînement
results = model.train(
    data=yaml_path,     # Votre dataset
    epochs=100,          # Nombre de passes sur le dataset
    imgsz=640,          # Taille des images (redimensionnement auto)
    batch=8,           # Taille du batch (ajuster selon votre VRAM)
    device=device,      # GPU ou CPU
    optimizer='auto',   # 'SGD', 'Adam', 'AdamW' ou 'auto'
    name='red_cube_run' # Nom du dossier de sauvegarde dans 'runs/'
)

print("Entraînement terminé.")

Démarrage du fine-tuning...
New https://pypi.org/project/ultralytics/8.3.241 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.239 🚀 Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/red_cube/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=models/yolo11n_base/yolo11n.pt, momentum=0.937, mos

/home/nathan/miniconda3/envs/oak/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
2025/12/25 21:52:44 INFO mlflow.tracking.fluent: Experiment with name '/Shared/Ultralytics' does not exist. Creating a new experiment.


MLflow: logging run_id(f8619342a4f5402f841565f8facaa904) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_run5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/50      2.16G     0.4866       3.17     0.9844         27        640: 100% ━━━━━━━━━━━━ 2/2 3.8s/it 7.6s<11.7s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 1.2s/it 1.2s
                   all          8          8    0.00333          1      0.213      0.207

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50      2.16G      0.537      3.245      1.029         26        640: 100% ━━━━━━━━━━━━ 2/2 1

In [14]:
# Évaluation du modèle sur le jeu de validation
metrics = model.val()

# Affichage des métriques clés
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")
print(f"Précision: {metrics.box.mp:.4f}")
print(f"Rappel (Recall): {metrics.box.mr:.4f}")

Ultralytics 8.3.239 🚀 Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5694.8±3333.1 MB/s, size: 1924.5 KB)
val: Scanning /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/labels.cache... 8 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 8/8 34.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.5it/s 0.4s
                   all          8          8    0.00333          1      0.995      0.939
Speed: 0.5ms preprocess, 9.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/val
mAP@50: 0.9950
mAP@50-95: 0.9393
Précision: 0.0033
Rappel (Recall): 1.0000


In [15]:
from pathlib import Path
import random

# Récupérer une image au hasard dans le dossier de validation
valid_images_path = Path('datasets/red_cube/valid/images')
test_image = random.choice(list(valid_images_path.glob('*.jpg')))

# Faire une prédiction avec le modèle fine-tuné
# conf=0.5 signifie qu'on ne garde que les détections sûres à >50%
results_predict = model.predict(source=test_image, conf=0.5, save=True)

print(f"Prédiction sauvegardée pour : {test_image.name}")
# L'image annotée sera sauvegardée dans runs/detect/predict/


image 1/1 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251210_085514.jpg: 640x480 (no detections), 25.5ms
Speed: 1.4ms preprocess, 25.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/predict
Prédiction sauvegardée pour : 20251210_085514.jpg


In [16]:
# On baisse le seuil de confiance (conf) à 0.10 pour voir même les détections faibles
# On augmente iou à 0.5 pour fusionner les boîtes qui se chevauchent
results_debug = model.predict(
    source='datasets/red_cube/valid/images', # On prend tout le dossier de valid
    conf=0.10,  # Seuil très bas pour tout voir
    iou=0.5,    # Nettoyage des doublons
    save=True,
    max_det=5   # On limite à 5 boîtes max par image pour éviter le spam visuel
)

print(f"Résultats sauvegardés dans : {results_debug[0].save_dir}")


image 1/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251209_190950.jpg: 480x640 (no detections), 26.7ms
image 2/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251210_085451.jpg: 640x480 (no detections), 3.0ms
image 3/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251210_085457.jpg: 640x480 (no detections), 2.8ms
image 4/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251210_085514.jpg: 640x480 (no detections), 2.8ms
image 5/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251210_085606.jpg: 640x480 (no detections), 2.8ms
image 6/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/20251210_085729.jpg: 480x640 (no detections), 3.0ms
image 7/8 /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/images/IMG_0408.JPG: 